# Måle inn punkt med GNSS
Vi skal måle gjennomsnitt på de forskjellige måtene vi har samlet inn data. Vi har en KML fil med data fra RTK'n, og et datasett fra telefon. 

## Gjennomsnitt ved RTK målinger
Først får vi ut koordinatene fra KML filen og lagrer det i et array.

In [49]:
import xml.etree.ElementTree as ET
import numpy as np


def extract_coordinates_kml_RTK (kml_file):
    tree = ET.parse(kml_file)
    root = tree.getroot()
    
    # Namespace er noe vi må ha med siden det ikke er en XML fil
    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Går igjennom alle punktene - må her også ha spesielt format pga kml-fil

    coordinates = []

    for placemark in root.findall('.//kml:Placemark', namespace):
            longitude = placemark.find('.//kml:SimpleData[@name="Longitude"]', namespace).text
            latitude = placemark.find('.//kml:SimpleData[@name="Latitude"]', namespace).text
            coordinates.append([float(longitude), float(latitude)])
    return coordinates

extract_coordinates_kml_RTK('RTK.kml')

[[10.77530275, 59.66589043],
 [10.77530274, 59.66589059],
 [10.7753027, 59.6658904],
 [10.77530269, 59.66589036],
 [10.77530294, 59.6658903],
 [10.77530266, 59.66589073],
 [10.77530263, 59.66589051]]

Så lager vi en funkjson for å beregne gjennomsnitt:

In [50]:

def average_coordinates(array_input):
    coordinates = np.array(array_input)

    # Må ha axis=0 slik at funksjonen ikke regner ut gjennomsnittet av alle punktene
    average_coordinates_point = np.mean(coordinates, axis=0)
    print('Gjennomsnittskoordinatet er:', average_coordinates_point)


Kjører denne funksjonen på punktene målt med RTK.

In [51]:
average_coordinates(extract_coordinates_kml_RTK('RTK.kml'))

Gjennomsnittskoordinatet er: [10.77530273 59.66589047]


## Gjennomsnitt ved mobil

Her har vi to ulike måter. Filen vi fikk fra Trygve i tilfelle NMEA dataen viser seg trøblete

### Trygves data

Trekker så ut punktene fra KML filen fra Trygves mobil, og lager ett array av de. Funksjonen blir veldig lik den forrige funksjonen, men med små endringer (Bryter litt med DNRY)

In [53]:
def extract_coordinates_kml_mobil (kml_file):
    tree = ET.parse(kml_file)
    root = tree.getroot()

    # Må bruke http://www.google.com/kml/ext/2.2 her i stedet for klm, siden gx:coord er google spesifik .klm
    namespace = {'gx': 'http://www.google.com/kml/ext/2.2'}

    coordinates = []
    for coord in root.findall('.//gx:coord', namespace):
        #understreken her var kul
        lon, lat, _ = map(float, coord.text.split())
        coordinates.append([lon, lat])

    return coordinates

Kjører så gjennomsnittsfunksjonen for å finne ut gjennomsnittet ved disse målingene

In [54]:
average_coordinates(extract_coordinates_kml_mobil('trygve_data.kml'))

Gjennomsnittskoordinatet er: [10.77532085 59.6658624 ]
